In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'test-281700'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://spotify_asr_dataset/


In [ ]:
bucket_name = 'spotify_asr_dataset'

#download metadata for episodes
!gsutil -m cp -r gs://{bucket_name}/metadata.tsv /content/

#download itunes data
!gsutil -m cp -r gs://{bucket_name}/episodes.csv /content/

Copying gs://spotify_asr_dataset/metadata.tsv...
| [1/1 files][112.2 MiB/112.2 MiB] 100% Done                                    
Operation completed over 1 objects/112.2 MiB.                                    
Copying gs://spotify_asr_dataset/episodes.csv...
/ [1/1 files][  1.2 GiB/  1.2 GiB] 100% Done  35.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.2 GiB.                                      


In [ ]:
import pandas as pd

metadata = pd.read_csv('metadata.tsv', sep='\t')
itunes_episodes = pd.read_csv('episodes.csv')
itunes_shows = pd.read_csv('shows.csv')

In [ ]:
metadata.head(2)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA


In [ ]:
itunes_episodes.head(2)

,id,show_id,title,link,guid,subtitle,description,summary,author,audio_url,audio_file_size,audio_mime_type,category,explicit,length,pub_date,keywords
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,https://www.podomatic.com/podcasts/teamosmnetw...,http://teamosmnetwork.podomatic.com/entry/2008...,DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,"<img src=""http://assets.podomatic.net/ts/e7/04...",DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,NaN,http://teamosmnetwork.podOmatic.com/enclosure/...,81099614,audio/mpeg,NaN,1,5571,2008-03-19T17:21:46,"2pac,97,akon,ali,amazing,andre,ashanti,autogra..."
1,000020d5-40ec-351e-aa7a-e8af775de245,16f67b61-d35c-38fa-9bfb-36146d2be45b,Back To Goa - Kameleon part 2 (Archaic Revival),http://max99.libsyn.com/back-to-goa-kameleon-p...,be56007c5636418ce74c58b75dade7f3,Special part 2 of 2 of Back To Goa Pangea spec...,"<p style=""margin: 0.0px 0.0px 0.0px 0.0px; fon...",NaN,NaN,http://traffic.libsyn.com/max99/Back_To_Goa_-_...,189289362,audio/mpeg,NaN,0,1243,2011-10-08T21:15:00,"to,back,trance,pangea,goa,psy,kameleon"


In [ ]:
itunes_shows.head(2)

,id,feed_url,title,subtitle,description,summary,author,email,link,language,explicit,image,category,subcategory,created_at,last_build_date
0,0002da5f-49a2-31ba-b44c-cdeabdf113cb,http://www.cbc.ca/podcasting/includes/dispatch...,Dispatches from CBC Radio,NaN,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio,podcasting@cbc.ca,http://www.cbc.ca/podcasting,en-ca,0,http://www.cbc.ca/podcasting/images/promo-disp...,News & Politics,Public Radio,2016-11-27T03:15:54,2016-11-27T03:15:54
1,00044845-beb8-35f3-99e2-01dbcfb6eb63,http://everydayissaturday.com/feed/podcast/,Motivation | Inspiration| Success with Entrepr...,Motivation - Inspiration - Success - Mindset -...,Sam Crowley delivers million dollar motivation...,Sam Crowley delivers million dollar motivation...,Sam Crowley,sam@everydayissaturday.com,http://everydayissaturday.com,en-us,0,http://everydayissaturday.com/wp-content/uploa...,Business,Business,2016-11-26T19:11:59,2016-11-21T14:19:34


In [ ]:
itunes_data = itunes_episodes.merge(itunes_shows, left_on='show_id', right_on='id')

In [ ]:
itunes_data.head(2)

,id_x,show_id,title_x,link_x,guid,subtitle_x,description_x,summary_x,author_x,audio_url,audio_file_size,audio_mime_type,category_x,explicit_x,length,pub_date,keywords,id_y,feed_url,title_y,subtitle_y,description_y,summary_y,author_y,email,link_y,language,explicit_y,image,category_y,subcategory,created_at,last_build_date
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,https://www.podomatic.com/podcasts/teamosmnetw...,http://teamosmnetwork.podomatic.com/entry/2008...,DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,"<img src=""http://assets.podomatic.net/ts/e7/04...",DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,NaN,http://teamosmnetwork.podOmatic.com/enclosure/...,81099614,audio/mpeg,NaN,1,5571,2008-03-19T17:21:46,"2pac,97,akon,ali,amazing,andre,ashanti,autogra...",8ab10fd5-1acb-330e-be52-beebf19f0703,http://teamosmnetwork.podomatic.com/rss2.xml,TEAM OSM NETWORK,IT'S ONLY ONE TEAM ON TOP,NaN,NaN,TEAM OSM NETWORK,NaN,NaN,en-us,0,http://teamosmnetwork.podomatic.com/images/def...,Music,NaN,2016-11-26T18:39:21,2016-11-26T18:39:21
1,330fff01-bf2f-365f-83f0-d5f3ea5d065d,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ 4SHO EAR O THE STREETZ RADIO VOL.3 SOUTHERN...,https://www.podomatic.com/podcasts/teamosmnetw...,http://teamosmnetwork.podomatic.com/entry/2008...,"1 THE BOSS RICK ROSS, T PAIN 2 GHETTO RICH RIC...","<img src=""http://assets.podomatic.net/ts/e7/04...","1 THE BOSS RICK ROSS, T PAIN 2 GHETTO RICH RIC...",NaN,http://teamosmnetwork.podOmatic.com/enclosure/...,60403106,audio/mpeg,NaN,1,4150,2008-03-03T20:33:37,"2pac,97,akon,ali,amazing,andre,ashanti,autogra...",8ab10fd5-1acb-330e-be52-beebf19f0703,http://teamosmnetwork.podomatic.com/rss2.xml,TEAM OSM NETWORK,IT'S ONLY ONE TEAM ON TOP,NaN,NaN,TEAM OSM NETWORK,NaN,NaN,en-us,0,http://teamosmnetwork.podomatic.com/images/def...,Music,NaN,2016-11-26T18:39:21,2016-11-26T18:39:21


In [ ]:
itunes_data = itunes_data[['id_x', 'id_y', 'title_x', 'summary_x', 'title_y']]
itunes_data.head(2)

,id_x,id_y,title_x,summary_x,title_y
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,TEAM OSM NETWORK
1,330fff01-bf2f-365f-83f0-d5f3ea5d065d,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ 4SHO EAR O THE STREETZ RADIO VOL.3 SOUTHERN...,"1 THE BOSS RICK ROSS, T PAIN 2 GHETTO RICH RIC...",TEAM OSM NETWORK


In [ ]:
combined = metadata.merge(itunes_data, left_on=['show_name','episode_name'], right_on = ['title_y','title_x'])

In [ ]:
combined.head(2)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,id_x,id_y,title_x,summary_x,title_y
0,spotify:show:7gozmLqbcbr6PScMjc0Zl4,Reply All,"""'A podcast about the internet' that is actual...",Gimlet,['en'],https://feeds.megaphone.fm/replyall,spotify:episode:01ugJcZ5cbSdvhrL92WXYW,"#24 Exit & Return, Part II","This week, we conclude Shulem Deen’s story. I...",23.184100,show_7gozmLqbcbr6PScMjc0Zl4,01ugJcZ5cbSdvhrL92WXYW,24f81324-ccb7-3f72-b454-2914bd2736e1,850417fa-904f-3cfc-8792-751d4b9f04b2,"#24 Exit & Return, Part II","This week, we conclude Shulem Deen’s story.&nb...",Reply All
1,spotify:show:7gozmLqbcbr6PScMjc0Zl4,Reply All,"""'A podcast about the internet' that is actual...",Gimlet,['en'],https://feeds.megaphone.fm/replyall,spotify:episode:0A6JZqMLAA6Kf8mDQXFVrr,#18 Silence And Respect,"In 2012, a woman named Lindsey Stone posted a ...",26.822517,show_7gozmLqbcbr6PScMjc0Zl4,0A6JZqMLAA6Kf8mDQXFVrr,36ac13ab-e84b-3cf1-9fac-a31de9a192c8,850417fa-904f-3cfc-8792-751d4b9f04b2,#18 Silence And Respect,"In 2012, a woman named Lindsey Stone posted a ...",Reply All


In [ ]:
len(combined)

165

In [ ]:
combined.iloc[100]['episode_description']

'If you want perfect stickiness, look to gecko feet - not a crayfish straightjacket. The Facts Surprisingly Awesome’s theme music is “How We Do” by Nicholas Britell. Our ad music is by Build Buildings. This episode was edited by Alex Blumberg and Caitlin Kenney, and produced by Rachel Ward and Kalila Holt, with help from Emma Jacobs, Jacob Cruz, Laura Sims, and was mixed by Andrew Dunn. Special thanks to Jody Roberts at the Chemical Heritage Foundation. Sponsors Mack Weldon Squarespace NatureBox'

In [ ]:
combined.iloc[100]['summary_x']

'If you want perfect stickiness, look to gecko feet - not a crayfish straightjacket.\n\nThe Facts\nSurprisingly Awesome’s theme music is “How We Do” by Nicholas Britell. Our ad music is by Build Buildings. This episode was edited by Alex Blumberg and Caitlin Kenney, and produced by Rachel Ward and Kalila Holt, with help from Emma Jacobs, Jacob Cruz, Laura Sims, and was mixed by Andrew Dunn. Special thanks to Jody Roberts at the Chemical Heritage Foundation.\n\nSponsors\nMack Weldon\nSquarespace\nNatureBox'

Desriptions are the exact same so there is no point of retraining, since i need the transcripts for training